In [2]:
import numpy as np 
import pandas as pd
import scipy.stats
from tqdm import tqdm_notebook

from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from xgboost import XGBRegressor

import optuna
from rdkit import Chem
from molfeat.calc import FPCalculator

def canonize_smiles(smiles):
    return Chem.MolToSmiles(Chem.MolFromSmiles(smiles))
    
import warnings
warnings.filterwarnings("ignore")

In [3]:
df = pd.read_csv('IrLumDB.csv')
test = pd.read_csv('Synthesized_complexes.csv')

In [4]:
df = df[~df['L1'].apply(lambda x: 'si' in x.lower())]
df = df[~df['L3'].apply(lambda x: 'si' in x.lower())]
df = df[~df['L1'].apply(lambda x: 'b' in x.lower())]
df = df[~df['L3'].apply(lambda x: 'b' in x.lower())]
df = df[df['L3'].apply(lambda x: len(x) > 5)]

df['L1_mol'] = df['L1'].apply(Chem.MolFromSmiles)
df['L2_mol'] = df['L2'].apply(Chem.MolFromSmiles)
df['L3_mol'] = df['L3'].apply(Chem.MolFromSmiles)
test['L1_mol'] = test['L1'].apply(Chem.MolFromSmiles)
test['L2_mol'] = test['L2'].apply(Chem.MolFromSmiles)
test['L3_mol'] = test['L3'].apply(Chem.MolFromSmiles)

df_ch2cl2 = df[df['Solvent'] == 'CH2Cl2']
df_ch2cl2.drop_duplicates(subset=['L1', 'L2', 'L3'], inplace=True)
df_ch2cl2.reset_index(drop=True, inplace=True)

In [9]:
def get_finger(fingerprints):
    """
    This function creates fingerprints from SMILES ligands. 
    The list of available fingerprints can be viewed: FPCalculator.available_fingerprints()
    """
    for f in fingerprints:
        calc = FPCalculator(f)
        df_ch2cl2[f'L1_{f}'] = df_ch2cl2['L1_mol'].apply(calc)
        df_ch2cl2[f'L2_{f}'] = df_ch2cl2['L2_mol'].apply(calc)
        df_ch2cl2[f'L3_{f}'] = df_ch2cl2['L3_mol'].apply(calc)
        df_ch2cl2[f'{f}'] = np.sum([df_ch2cl2[f'L1_{f}'], df_ch2cl2[f'L2_{f}'], df_ch2cl2[f'L3_{f}']], axis=0)
        test[f'L1_{f}'] = test['L1_mol'].apply(calc)
        test[f'L2_{f}'] = test['L2_mol'].apply(calc)
        test[f'L3_{f}'] = test['L3_mol'].apply(calc)
        test[f'{f}'] = np.sum([test[f'L1_{f}'], test[f'L2_{f}'], test[f'L3_{f}']], axis=0)

    df_qy = df_ch2cl2[~df_ch2cl2['PLQY'].isna()]
    df_qy = df_qy[df_qy['PLQY_in_train'] != 0]
    X, y = df_qy[fingerprints].to_numpy(), df_qy['PLQY'].to_numpy()
    X = np.array([np.hstack(i) for i in X])
    df_result = pd.DataFrame()
    for d in X:
        df_result = pd.concat([df_result, pd.DataFrame(d).T])
    X = df_result.to_numpy()
    print(X.shape)
    
    return X, y

In [11]:
X, y = get_finger(['ecfp'])

(724, 2048)


In [12]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=42, shuffle=True)

In [13]:
kf = KFold(n_splits=10, shuffle=True, random_state=42)

# Optuna

In [ ]:
def objective_catboost(trial):
    params = {
        "iterations": 1000,
        "learning_rate": trial.suggest_float("learning_rate", 2*1e-3, 0.2, log=True),
        "depth": trial.suggest_int("depth", 1, 10),
        "subsample": trial.suggest_float("subsample", 0.05, 1.0),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.05, 1.0),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 1, 100),
    }

    model = CatBoostRegressor(**params, silent=True, random_state=42)
    model.fit(X_train, y_train)
    predictions = model.predict(X_val)
    rmse = mean_squared_error(y_val, predictions, squared=False)
    return rmse

In [ ]:
def objective_lgbm(trial):
    params = {
        "objective": "regression",
        "metric": "rmse",
        "n_estimators": 1000,
        "verbosity": -1,
        "bagging_freq": 1,
        "learning_rate": trial.suggest_float("learning_rate", 2*1e-3, 0.2, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 2, 2**10),
        "subsample": trial.suggest_float("subsample", 0.05, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.05, 1.0),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 1, 100),
    }
    model = LGBMRegressor(**params, silent=True, random_state=42)
    model.fit(X_train, y_train)
    predictions = model.predict(X_val)
    rmse = mean_squared_error(y_val, predictions, squared=False)
    return rmse

In [ ]:
def objective_xgboost(trial):
    params = {
        "objective": "reg:squarederror",
        "n_estimators": 1000,
        "verbosity": 0,
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.1, log=True),
        "max_depth": trial.suggest_int("max_depth", 1, 10),
        "subsample": trial.suggest_float("subsample", 0.05, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.05, 1.0),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 20),
    }

    model = XGBRegressor(**params)
    model.fit(X_train, y_train, verbose=False)
    predictions = model.predict(X_val)
    rmse = mean_squared_error(y_val, predictions, squared=False)
    return rmse

In [ ]:
def objective_svr(trial):
    params = {
        "C": trial.suggest_float('C', 1, 1000, log=True),
        "epsilon": trial.suggest_float('epsilon', 1e-3, 1, log=True),
    }

    model = SVR(**params)
    model.fit(X_train, y_train)
    predictions = model.predict(X_val)
    rmse = mean_squared_error(y_val, predictions, squared=False)
    return rmse

In [ ]:
def objective_knn(trial):
    params = {
        "n_neighbors": trial.suggest_int("n_neighbors", 1, 100, log=True),
    }

    model = KNeighborsRegressor(**params, random_state=42)
    model.fit(X_train, y_train)
    predictions = model.predict(X_val)
    rmse = mean_squared_error(y_val, predictions, squared=False)
    return rmse

# Functions for training and validating models

In [22]:
def mean_confidence_interval(data, metric, confidence=0.95):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
    m = round(m ,3)
    h = round(h, 3)
    print(f'{metric}: {m} ± {h}')

In [15]:
def cv10(model):
    mae_result = []
    rmse_result = []
    r2_result = []
    for train, val in tqdm_notebook(kf.split(X, y)):
        model.fit(X[train], y[train])
        y_pred_val = model.predict(X[val])
        mae_result.append(mean_absolute_error(y[val], y_pred_val))
        rmse_result.append(mean_squared_error(y[val], y_pred_val, squared=False))
        r2_result.append(r2_score(y[val], y_pred_val))
    mean_confidence_interval(mae_result, 'MAE')
    mean_confidence_interval(rmse_result, 'RMSE')
    mean_confidence_interval(r2_result, 'R2')

# Find best params

In [ ]:
#сatboost
сatboost_study = optuna.create_study(direction='minimize')
сatboost_study.optimize(objective_catboost, n_trials=30)
catboost_study.best_params  

In [17]:
cat_bp = {'learning_rate': 0.04153336533280499,
 'depth': 10,
 'subsample': 0.9132677097127234,
 'colsample_bylevel': 0.540038955308394,
 'min_data_in_leaf': 61,
'n_estimators':1000}

In [18]:
model_cat = CatBoostRegressor(**cat_bp, random_state=42, silent=True)

In [23]:
cv10(model_cat)

0it [00:00, ?it/s]

MAE: 0.129 ± 0.009
RMSE: 0.18 ± 0.015
R2: 0.583 ± 0.043


In [ ]:
#xgboost
xgboost_study = optuna.create_study(direction='minimize')
xgboost_study.optimize(objective_xgboost, n_trials=30)
xgboost_study.best_params  

In [24]:
xgb_bp = {'learning_rate': 0.06238679289783574,
 'max_depth': 7,
 'subsample': 0.9982371412074009,
 'colsample_bytree': 0.1637075927345035,
 'min_child_weight': 1,
 'objective': 'reg:squarederror',
 'n_estimators': 1000,
 'verbosity': 0}

In [25]:
model_xgb = XGBRegressor(**xgb_bp, random_state=42, silent=True)

In [26]:
cv10(model_xgb)

0it [00:00, ?it/s]

MAE: 0.129 ± 0.01
RMSE: 0.185 ± 0.013
R2: 0.561 ± 0.035


In [ ]:
#lightgbm
lightgbm_study = optuna.create_study(direction='minimize')
lightgbm_study.optimize(objective_lgbm, n_trials=30)
lightgbm_study.best_params

In [27]:
lgbm_bp = {'learning_rate': 0.0312578769878087,
 'num_leaves': 367,
 'subsample': 0.5808481639196008,
 'colsample_bytree': 0.28534206471458423,
 'min_data_in_leaf': 1,
'objective': 'regression',
 'metric': 'rmse',
 'n_estimators': 100,
 'verbosity': -1,
 'bagging_freq': 1}

In [28]:
model_lgbm = LGBMRegressor(**lgbm_bp, random_state=42)

In [29]:
cv10(model_lgbm)

0it [00:00, ?it/s]

[LightGBM] [Warning] min_data_in_leaf is set=1, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=1
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] min_data_in_leaf is set=1, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=1
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] min_data_in_leaf is set=1, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=1
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] min_data_in_leaf is set=1, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=1
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] min_data_in_leaf is set=1, min_child_samples=20 will be ignored. Cu

In [ ]:
#svr
svr_study = optuna.create_study(direction='minimize')
svr_study.optimize(objective_svr, n_trials=30)
svr_study.best_params 

In [30]:
svr_bp = {'C': 1.1803186568111252, 'epsilon': 0.004389660374508496}

In [31]:
model_svr = SVR(**svr_bp)

In [32]:
cv10(model_svr)

0it [00:00, ?it/s]

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
MAE: 0.128 ± 0.01
RMSE: 0.181 ± 0.015
R2: 0.577 ± 0.074


In [ ]:
#knn
knn_study = optuna.create_study(direction='minimize')
knn_study.optimize(objective_knn, n_trials=30)
knn_study.best_params 

In [34]:
knn_bp = {'n_neighbors': 2}

In [35]:
model_knn = KNeighborsRegressor(**knn_bp)

In [36]:
cv10(model_knn)

0it [00:00, ?it/s]

MAE: 0.142 ± 0.009
RMSE: 0.207 ± 0.014
R2: 0.441 ± 0.099
